In [1]:
BATCH_SIZE = 8
IMAGE_SIZE = 64
SEED = 42
AUTOENCODER_EPOCHS = 3
CLASSIFIER_EPOCHS = 20
num_classes = 5 
class_names = ["cardboard", "glass", "metal", "paper", "plastic"]


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Flatten, Dense, Rescaling

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from tensorflow.keras import layers, losses
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model
from scikeras.wrappers import KerasClassifier
import tensorflow_datasets as tfds



c:\Users\bgd45\Desktop\Data mining project\data_mining_venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
dataset_dir = "./dataset"
train, validate = tf.keras.utils.image_dataset_from_directory(
    dataset_dir,
    labels='inferred',
    label_mode='int',
    color_mode='rgb',
    batch_size=BATCH_SIZE,
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    shuffle=True,
    seed=SEED,
    subset="both",
    validation_split=0.2,
    interpolation='bilinear',

)



Found 2390 files belonging to 5 classes.
Using 1912 files for training.
Using 478 files for validation.


In [4]:
normalization_layer = tf.keras.layers.Rescaling(1./255)
def change_inputs(images, labels):
    x = tf.image.resize(normalization_layer(images), [IMAGE_SIZE, IMAGE_SIZE], method=tf.image.ResizeMethod.GAUSSIAN)
    return x, x

normalized_ds = train.map(change_inputs)

# Autoencoder Model
def create_autoencoder_model():
    input_layer = tf.keras.layers.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))

    x = Conv2D(64, (3, 3), activation='relu', padding='same')(input_layer)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    encoded = MaxPooling2D((2, 2), padding='same')(x)

    x = tf.keras.layers.Dropout(0.5)(encoded) 
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

    autoencoder = Model(input_layer, decoded)
    autoencoder.compile(optimizer='adam', loss='mse')
    return autoencoder

# Training Autoencoder
autoencoder_model = create_autoencoder_model()
history_autoencoder = autoencoder_model.fit(normalized_ds, epochs=AUTOENCODER_EPOCHS)

# Encode data using the trained autoencoder
encoded_data = autoencoder_model.predict(normalized_ds)
print(f"Encoded Data Shape: {encoded_data.shape}")








Epoch 1/3

239/239 [==============================] - 29s 117ms/step - loss: 0.0105
Epoch 2/3
239/239 [==============================] - 28s 118ms/step - loss: 0.0038
Epoch 3/3
239/239 [==============================] - 9s 38ms/step
Encoded Data Shape: (1912, 64, 64, 3)


In [5]:
def create_classifier_model(autoencoder_model):
    # Use the encoder part of the autoencoder as a feature extractor
    encoder_output = autoencoder_model.layers[7].output  # Assuming the encoder output is at index 7, adjust if needed

    # Add classification layers on top of the encoder output
    x = layers.Flatten()(encoder_output)
    x = layers.Dense(128, activation='relu')(x)
    output_layer = layers.Dense(num_classes, activation='softmax')(x)

    classifier_model = Model(autoencoder_model.input, output_layer)
    classifier_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return classifier_model

# Pass your autoencoder model when creating the classifier
classifier_model = create_classifier_model(autoencoder_model)
history_classifier = classifier_model.fit(train, epochs=CLASSIFIER_EPOCHS, validation_data=validate)

Epoch 1/20

239/239 [==============================] - 45s 183ms/step - loss: 55.7109 - accuracy: 0.2615 - val_loss: 1.6079 - val_accuracy: 0.2510
Epoch 2/20
239/239 [==============================] - 47s 196ms/step - loss: 1.5239 - accuracy: 0.3405 - val_loss: 1.6029 - val_accuracy: 0.2552
Epoch 3/20
239/239 [==============================] - 53s 222ms/step - loss: 1.4705 - accuracy: 0.3876 - val_loss: 1.5826 - val_accuracy: 0.3013
Epoch 4/20
214/239 [=========================>....] - ETA: 5s - loss: 1.4963 - accuracy: 0.3668

In [ ]:
validate_normalized = validate.map(change_inputs)
validate_list = list(validate_normalized.as_numpy_iterator())
images_and_labels = list(validate.as_numpy_iterator())
# Predict labels for validation images
predictions = classifier_model.predict(validate)
predicted_labels = np.argmax(predictions, axis=1)
print(predicted_labels)
y = np.concatenate([y for x, y in validate], axis=0)

# Display a few validation images along with their true and predicted labels
plt.figure(figsize=(10, 10))
for i in range(15):  # Adjust the number of images you want to display
    plt.subplot(4, 4, i + 1)
    
    # Access the batch of images and labels
    image_batch = images_and_labels[i][0]
    label_batch = images_and_labels[i][1]
    
    # Access the ith image and label from the batch
    image = image_batch[i]
    label = label_batch[i]
    
    plt.imshow(image.astype(np.uint8) / 255)
    plt.title(f"True: {class_names[label]}\nPredicted: {class_names[predicted_labels[i]]}")
    plt.axis('off')

plt.show()


In [ ]:
normalized_validate = validate.map(change_inputs)
evaluation_results = autoencoder.evaluate(normalized_validate)
print("Validation Loss:", evaluation_results)


In [ ]:
def get_validation_batch(dataset, batch_size=5):
    for images, _ in dataset.take(1):
        return images[:batch_size]

# Get a batch of images
sample_images = get_validation_batch(normalized_validate)

# Generate reconstructions
reconstructed_images = autoencoder.predict(sample_images)

# Visualize original vs. reconstructed images
for i in range(len(sample_images)):
    plt.subplot(2, len(sample_images), i + 1)
    plt.imshow(sample_images[i])
    plt.title("Original")
    plt.axis("off")

    plt.subplot(2, len(sample_images), i + 1 + len(sample_images))
    plt.imshow(reconstructed_images[i])
    plt.title("Recon")
    plt.axis("off")

plt.show()